In [1]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Additional Imports for ML model

from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Import and read the data
fire_df = pd.read_csv("Resources/FW_Veg_Rem_Combined.csv")
fire_df.head(10)

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633
5,5,5,NaN,1.0,B,Debris Burning,32.063900,-82.417800,GA,3/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.139643
6,6,6,NaN,5.2,B,Debris Burning,31.316978,-83.393649,GA,1/10/2009,...,0.000000,79.896679,73.431818,0.000000,0.000000,26.0,0.0,0.0,0.0,0.148904
7,7,7,NaN,1.0,B,Campfire,30.904720,-93.557500,TX,11/12/2005,...,2.148857,72.899478,75.061381,77.924623,70.732911,28.4,27.5,1.2,55.4,0.241894
8,8,8,NaN,1.0,B,Arson,35.900310,-92.061180,AR,8/8/2007,...,2.103090,68.319022,67.575419,65.077844,60.196858,6.6,3.3,0.0,46.4,0.224629
9,9,9,CRAZY,1.0,B,Miscellaneous,48.839400,-99.718500,ND,4/19/2004,...,0.000000,64.606509,55.943038,54.337838,0.000000,12.3,1.8,0.0,0.0,0.291683


In [3]:
fire_df.dtypes

Unnamed: 0            int64
Unnamed: 0.1          int64
fire_name            object
fire_size           float64
fire_size_class      object
stat_cause_descr     object
latitude            float64
longitude           float64
state                object
disc_clean_date      object
cont_clean_date      object
discovery_month      object
disc_date_final      object
cont_date_final      object
putout_time          object
disc_date_pre        object
disc_pre_year         int64
disc_pre_month       object
wstation_usaf        object
dstation_m          float64
wstation_wban         int64
wstation_byear        int64
wstation_eyear        int64
Vegetation            int64
fire_mag            float64
weather_file         object
Temp_pre_30         float64
Temp_pre_15         float64
Temp_pre_7          float64
Temp_cont           float64
Wind_pre_30         float64
Wind_pre_15         float64
Wind_pre_7          float64
Wind_cont           float64
Hum_pre_30          float64
Hum_pre_15          

In [4]:
!pip install pandas-profiling[notebook]

  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27085 sha256=298250bb7946de1baaf9651fa968509958ab18fa17c47256d2d0e7dea16cf646
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\70\e1\52\5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for phik: filename=phik-0.11.2-py3-none-any.whl size=1107415 sha256=16660c9ce448ad8088db61bc2c1eab65fa40e462b2e57ed4585feb44e9e98faf
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\40\98\a3\b654f24edcdcdb87d1f70d65a506fcfdf15289db129c594bcd
Successfully built htmlmin phik


In [5]:
from pandas_profiling import ProfileReport

In [6]:
profile = ProfileReport(fire_df, title="Pandas Profiling Report")

In [7]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/56 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
fire_df.drop(['Unnamed: 0.1','fire_name','cont_clean_date','disc_date_final','cont_date_final','putout_time','disc_clean_date','disc_pre_month'], axis=1, inplace=True)
fire_df.head()

KeyError: "['Unnamed: 0.1' 'fire_name' 'cont_clean_date' 'disc_date_final'\n 'cont_date_final' 'putout_time' 'disc_clean_date' 'disc_pre_month'] not found in axis"

In [11]:
fire_df.head()

,Unnamed: 0,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,discovery_month,disc_date_pre,disc_pre_year,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,Feb,1/12/2007,2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,3.0,B,Arson,35.038330,-87.610000,TN,Dec,11/11/2006,2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,60.0,C,Arson,34.947800,-88.722500,MS,Feb,1/30/2004,2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,1.0,B,Debris Burning,39.641400,-119.308300,NV,Jun,5/7/2005,2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,2.0,B,Miscellaneous,30.700600,-90.591400,LA,Sep,8/23/1999,1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [14]:
fire_df.isnull().count()

Unnamed: 0          55367
fire_size           55367
fire_size_class     55367
stat_cause_descr    55367
latitude            55367
longitude           55367
state               55367
discovery_month     55367
disc_date_pre       55367
disc_pre_year       55367
wstation_usaf       55367
dstation_m          55367
wstation_wban       55367
wstation_byear      55367
wstation_eyear      55367
Vegetation          55367
fire_mag            55367
weather_file        55367
Temp_pre_30         55367
Temp_pre_15         55367
Temp_pre_7          55367
Temp_cont           55367
Wind_pre_30         55367
Wind_pre_15         55367
Wind_pre_7          55367
Wind_cont           55367
Hum_pre_30          55367
Hum_pre_15          55367
Hum_pre_7           55367
Hum_cont            55367
Prec_pre_30         55367
Prec_pre_15         55367
Prec_pre_7          55367
Prec_cont           55367
remoteness          55367
dtype: int64

In [15]:
fire_df.rename(columns={'stat_cause_descr':'fire_cause', 'disc_pre_year':'discovery_year', 'disc_date_pre':'discovery_date'}, inplace=True)
fire_df.head()

,Unnamed: 0,fire_size,fire_size_class,fire_cause,latitude,longitude,state,discovery_month,discovery_date,discovery_year,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,Feb,1/12/2007,2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,3.0,B,Arson,35.038330,-87.610000,TN,Dec,11/11/2006,2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,60.0,C,Arson,34.947800,-88.722500,MS,Feb,1/30/2004,2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,1.0,B,Debris Burning,39.641400,-119.308300,NV,Jun,5/7/2005,2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,2.0,B,Miscellaneous,30.700600,-90.591400,LA,Sep,8/23/1999,1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [18]:
fire_df.drop(['weather_file'], axis=1, inplace=True)
fire_df.head()

,Unnamed: 0,fire_size,fire_size_class,fire_cause,latitude,longitude,state,discovery_month,discovery_date,discovery_year,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,Feb,1/12/2007,2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,3.0,B,Arson,35.038330,-87.610000,TN,Dec,11/11/2006,2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,60.0,C,Arson,34.947800,-88.722500,MS,Feb,1/30/2004,2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,1.0,B,Debris Burning,39.641400,-119.308300,NV,Jun,5/7/2005,2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,2.0,B,Miscellaneous,30.700600,-90.591400,LA,Sep,8/23/1999,1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [19]:
fire_df.to_csv('Resources/wildfire_data.csv', index=False)